In [1]:
#Matrices
import numpy as np 
import pandas as pd 

#Stats
import scipy
import scipy.stats as st
from scipy.optimize import fmin
from scipy import integrate
from scipy.stats.mstats import mquantiles
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import statsmodels.api as sm 
import statsmodels.formula.api as smf 
from itertools import combinations 
from patsy import dmatrix
#!pip install mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector


#Plotting
import matplotlib.pyplot as plt
import seaborn as sns

#Misc
#!pip install stargazer
from stargazer.stargazer import Stargazer #I also have the .py in the folder (in case it fails to install or run)

In [2]:
def dummy_levels(data, cat_col, ref_cat):
    cats = list(data[cat_col].unique()) #unique values in the categorical column
    # Remove ref_cat from the array and then insert it at the beginning of the array. 
    #We do this because the first element of the array will be the reference
    cats.remove(ref_cat)
    cats.insert(0, ref_cat) 
    
    return cats

In [15]:
data_bank = pd.read_csv("bank-additional-full.csv", sep = ",") #load data
data_bank.columns = data_bank.columns.str.replace(".", "_") #avoid dots in python
data_bank_dictionary = {"******** CLIENT DATA ********": 
                        {"age": "years", "job": "type of job", "marital": "marital status", "education": "education", 
                         "default": "has credit default?", "housing":"has housing loan", "loan":"has personal loan?"},
                       "******** LAST CONTACT CURRENT CAMPAIGN ********": 
                        {"contact": "contact communication type", "month": "last contact month of year",
                        "day_of_week": "last contact day of the week", 
                         "duration": "last contact duration, in seconds. Important note: this attribute affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model"},
                       "******** OTHER ********": 
                        {"campaign": "number of contacts performed during this campaign and for this client",
                        "pdays": "days that passed by after last contact from a previous campaign (999 means not previously contacted)",
                        "previous": "number of contacts performed before this campaign and for this client",
                        "poutcome": "outcome of the previous marketing campaign"},
                       "******** SOCIOECONOMIC ********": 
                        {"emp.var.rate": "employment variation rate - quarterly indicator",
                        "cons.price.idx": "consumer price index - monthly indicator",
                        "cons.conf.idx": "consumer confidence index - monthly indicator",
                        "euribor3m": "euribor 3 month rate - daily indicator",
                        "nr.employed": "number of employees - quarterly indicator"},
                       "******** DEPENDENT/ENDOGENOUS VARIABLE ********": 
                        {"y": "has the client subscribed a term deposit?"}}
print("VARIABLE MEANINGS (BY TYPE):\n")
data_bank_dictionary

VARIABLE MEANINGS (BY TYPE):



{'******** CLIENT DATA ********': {'age': 'years',
  'job': 'type of job',
  'marital': 'marital status',
  'education': 'education',
  'default': 'has credit default?',
  'housing': 'has housing loan',
  'loan': 'has personal loan?'},
 '******** LAST CONTACT CURRENT CAMPAIGN ********': {'contact': 'contact communication type',
  'month': 'last contact month of year',
  'day_of_week': 'last contact day of the week',
  'duration': "last contact duration, in seconds. Important note: this attribute affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model"},
 '******** OTHER ********': {'campaign': 'number of contacts performed during this campaign and for this client',
  'pdays': 'days that passed by after last contact from a previ

In [16]:
data_bank.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp_var_rate      float64
cons_price_idx    float64
cons_conf_idx     float64
euribor3m         float64
nr_employed       float64
y                  object
dtype: object

In [17]:
# Get the names of the numerical columns
numerical_cols = data_bank.select_dtypes(include=['number']).columns
print(numerical_cols)

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed'],
      dtype='object')


In [18]:
# Get the names of the categorical columns
categorical_cols = data_bank.select_dtypes(exclude=['number']).columns
print(categorical_cols)

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'day_of_week', 'poutcome', 'y'],
      dtype='object')


In [7]:
#Center numerical variables. 
#It facilitates interpretability + algorithm optimization
data_bank.loc[:, numerical_cols] = st.zscore(data_bank.loc[:, numerical_cols]) #center numerical variables
data_bank.loc[:, numerical_cols]

C:\Users\L03552670\AppData\Local\Temp\ipykernel_22652\1987726547.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 1.53303429  1.62899323 -0.29018564 ...  1.53303429  0.38152696
  3.26029527]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_bank.loc[:, numerical_cols] = st.zscore(data_bank.loc[:, numerical_cols]) #center numerical variables
C:\Users\L03552670\AppData\Local\Temp\ipykernel_22652\1987726547.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.01047142 -0.42150051 -0.12451981 ... -0.26722482  0.70856893
 -0.07438021]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_bank.loc[:, numerical_cols] = st.zscore(data_bank.loc[:, numerical_cols]) #center numerical variables
C:\Users\L03552670\AppData\Local\Temp\ipykernel_22652\1987726547.py:3:

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,1.533034,0.010471,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680
1,1.628993,-0.421501,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680
2,-0.290186,-0.124520,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680
3,-0.002309,-0.413787,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680
4,1.533034,0.187888,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680
...,...,...,...,...,...,...,...,...,...,...
41183,3.164336,0.292025,-0.565922,0.195414,-0.349494,-0.752343,2.058168,-2.224953,-1.495186,-2.815697
41184,0.573445,0.481012,-0.565922,0.195414,-0.349494,-0.752343,2.058168,-2.224953,-1.495186,-2.815697
41185,1.533034,-0.267225,-0.204909,0.195414,-0.349494,-0.752343,2.058168,-2.224953,-1.495186,-2.815697
41186,0.381527,0.708569,-0.565922,0.195414,-0.349494,-0.752343,2.058168,-2.224953,-1.495186,-2.815697


In [8]:
#Dummy coding of categorical data
#This is necessary so that all variables are numerical 0,1 (regressions work with numbers)
#For instance, say a column has three categories: "red", "green", "blue"
#We select blue as reference, and we put 0 and 1 to red and green i.e. two columns
#In the column "red", I put 1 whenever there is a red, 0 otherwise.
#In the columns "green", I put 1 whenever there is a green, 0 otherwise
#Note that blue is just when both red and green columns have 0s, 
#so we do not need a new column, just know which is the reference category

#Let's begin by defining the reference categories
refs = {'job':'unemployed',
       'marital': 'single',
       'education': 'university.degree',
       'default': 'no',
       'housing': 'no',
       'loan': 'no',
       'contact': 'telephone',
       'month': 'dec',
       'day_of_week': 'mon',
       'poutcome': 'failure'}

#Now let's put the references in a vector along with the other categories
print("The reference is always first\n\n")
levels = []
for key, value in refs.items():
    l = dummy_levels(data_bank, key, value)
    levels.append(l)
    print(key, l, "\n")
 


The reference is always first


job ['unemployed', 'housemaid', 'services', 'admin.', 'blue-collar', 'technician', 'retired', 'management', 'self-employed', 'unknown', 'entrepreneur', 'student'] 

marital ['single', 'married', 'divorced', 'unknown'] 

education ['university.degree', 'basic.4y', 'high.school', 'basic.6y', 'basic.9y', 'professional.course', 'unknown', 'illiterate'] 

default ['no', 'unknown', 'yes'] 

housing ['no', 'yes', 'unknown'] 

loan ['no', 'yes', 'unknown'] 

contact ['telephone', 'cellular'] 

month ['dec', 'may', 'jun', 'jul', 'aug', 'oct', 'nov', 'mar', 'apr', 'sep'] 

day_of_week ['mon', 'tue', 'wed', 'thu', 'fri'] 

poutcome ['failure', 'nonexistent', 'success'] 



In [24]:
# Define the endogenous variable. 
y = data_bank['y'].replace({'no': 0, 'yes': 1}) #we need to replace strings with 0 and 1

#Now the regression. Lasso will the reduce the number of variables
formula = "~ age + duration + campaign + pdays + " \
          "previous + emp_var_rate + cons_price_idx + " \
          "cons_conf_idx + euribor3m + nr_employed + " \
          "job + marital + education + default + housing + " \
          "loan + contact + month + day_of_week + poutcome"
dm = dmatrix(formula, data_bank)  #design matrix
X = np.asarray(dm)
dm
#dc = dmatrix("~ C(job, levels=jobs)", data_bank) #Dummy coding of job
#print("NUMBER OF ORIGINAL JOB CATEGORIES:\n", len(jobs))
#print("\nNUMBER OF JOB DUMMIES:\n", dc.shape[1]-1) #minus one because we do not count the intercept
#print("\nDUMMY COLUMNS (NOTE THAT THE REFERENCE IS NOT PRESENT)\n", dc.design_info.column_names)
#print("\nDUMMY CODES\nCOLUMNS AS ORGANIZED IN jobs (WITHOUT THE REFERENCE)\n", dc[:, 1:]) #Each column is a job (without the reference). There is a one if the job is there, else zero

#print("\nEXAMPLE. IN DUMMY CODES THE FIRST COLUMN IS FOR HOUSEMAID.\n THE FIRST ROW IS A 1 BECAUSE IN data_bank['job'] THE FIRST ROW IS HOUSEMAID\n", data_bank.loc[:, "job"])

C:\Users\L03552670\AppData\Local\Temp\ipykernel_22652\2467925629.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = data_bank['y'].replace({'no': 0, 'yes': 1}) #we need to replace strings with 0 and 1


DesignMatrix with shape (41188, 54)
  Columns:
    ['Intercept',
     'job[T.blue-collar]',
     'job[T.entrepreneur]',
     'job[T.housemaid]',
     'job[T.management]',
     'job[T.retired]',
     'job[T.self-employed]',
     'job[T.services]',
     'job[T.student]',
     'job[T.technician]',
     'job[T.unemployed]',
     'job[T.unknown]',
     'marital[T.married]',
     'marital[T.single]',
     'marital[T.unknown]',
     'education[T.basic.6y]',
     'education[T.basic.9y]',
     'education[T.high.school]',
     'education[T.illiterate]',
     'education[T.professional.course]',
     'education[T.university.degree]',
     'education[T.unknown]',
     'default[T.unknown]',
     'default[T.yes]',
     'housing[T.unknown]',
     'housing[T.yes]',
     'loan[T.unknown]',
     'loan[T.yes]',
     'contact[T.telephone]',
     'month[T.aug]',
     'month[T.dec]',
     'month[T.jul]',
     'month[T.jun]',
     'month[T.mar]',
     'month[T.may]',
     'month[T.nov]',
     'month[T.oct]',


In [ ]:
Div_l = ['E', 'W']
dm = dmatrix("~ 1 + C(League, levels=League_l) + C(Division, levels=Div_l) + C(NewLeague, levels=League_l)", 
             Hitters) #Dummy coding of categorical variables
print(dm.design_info.column_names) #to change manually to more readable names

In [ ]:
#Dummy coding of categorical data

In [ ]:
Hitters = Hitters.dropna().reset_index(drop=True)
non_cat_cols = np.append(np.array(Hitters.columns[0:13]), Hitters.columns[15:19]) #numerical columns
Hitters.loc[:, non_cat_cols] = st.zscore(Hitters.loc[:, non_cat_cols]) #center numerical variables

#Dummy coding of categorical data
League_l = ['N','A'] #first element will be the reference 0
Div_l = ['E', 'W']
dm = dmatrix("~ 1 + C(League, levels=League_l) + C(Division, levels=Div_l) + C(NewLeague, levels=League_l)", 
             Hitters) #Dummy coding of categorical variables
print(dm.design_info.column_names) #to change manually to more readable names

#Concatenate dummy coded variables with non-categorical variables
Hitters_d = pd.concat([pd.DataFrame(dm, columns=["Intercept", "League_1_American", "Division_1_West", "New_League_1_American"]), 
                       Hitters.loc[:, non_cat_cols]], axis = 1)
Hitters_d.drop(columns=['Intercept'], inplace = True)
print(Hitters_d.shape)
Hitters_d.dtypes

X = Hitters_d.loc[:, Hitters_d.columns[0:19]] #Predictors
y = Hitters_d.loc[:, "Salary"] #variable to predict